In [5]:
import cv2
import matplotlib.pyplot as plt

In [6]:
import numpy as np

In [46]:
img_path = "01_on.jpg"

In [47]:
image = cv2.imread(img_path)
lower_reso = cv2.pyrDown(image)
higher_reso = cv2.pyrUp(image)
cv2.imwrite(f'lower_reso_{img_path}', lower_reso)
cv2.imwrite(f'higher_reso_{img_path}', higher_reso)

True

In [48]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2.imwrite(f'hsv_image_{img_path}', hsv_image)

True

In [49]:
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
cv2.imwrite(f'blurred_image_{img_path}', blurred_image)

True

In [50]:
image = cv2.imread(img_path,0)
equalized_image = cv2.equalizeHist(image)
cv2.imwrite(f'equalized_image_{img_path}', equalized_image)

True

In [51]:
edges = cv2.Canny(image, 100, 200)
cv2.imwrite(f'edges_{img_path}', edges)

True

In [52]:
kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(image, kernel, iterations=1)
erosion = cv2.erode(image, kernel, iterations=1)
cv2.imwrite(f'dilation_{img_path}', dilation)
cv2.imwrite(f'erosion_{img_path}', erosion)

True

In [53]:
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
cv2.imwrite(f'binary_image_{img_path}', binary_image)

True

In [54]:
# 빛 반사 제거
def homomorphic_filter(image, d0=30, c=1.0, gH=2.0, gL=0.5):
    # Convert image to log domain
    image_log = np.log1p(np.array(image, dtype='float') / 255)
    
    # Perform DFT
    dft = cv2.dft(np.float32(image_log), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    
    # Create a high pass filter
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for u in range(rows):
        for v in range(cols):
            duv = np.sqrt((u - crow) ** 2 + (v - ccol) ** 2)
            mask[u, v] = (gH - gL) * (1 - np.exp(-c * (duv ** 2 / d0 ** 2))) + gL

    # Apply mask and inverse DFT
    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])
    
    # Convert back to spatial domain
    img_homomorphic = np.expm1(img_back)
    
    # Clip the values to prevent overflow and normalize
    img_homomorphic = np.clip(img_homomorphic, 0, None)
    img_homomorphic = (img_homomorphic - np.amin(img_homomorphic)) / (np.amax(img_homomorphic) - np.amin(img_homomorphic))
    img_homomorphic = (img_homomorphic * 255).astype(np.uint8)
    
    return img_homomorphic

In [55]:
# # 흑백이미지 대상
# image = cv2.imread(img_path, 0)  # Load as grayscale
# homomorphic_image = homomorphic_filter(image, d0=30, c=1.0, gH=1.5, gL=0.8)
# cv2.imwrite(f'homomorphic_image_{img_path}', homomorphic_image)

In [56]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
equalized_image = clahe.apply(image)
cv2.imwrite(f'equalized_image_{img_path}', equalized_image)

True

In [57]:
def single_scale_retinex(image, sigma):
    retinex = np.log10(image + 1) - np.log10(cv2.GaussianBlur(image, (0, 0), sigma) + 1)
    return retinex

image = cv2.imread(img_path) / 255.0
sigma = 15
retinex_image = single_scale_retinex(image, sigma)
retinex_image = (retinex_image - np.min(retinex_image)) / (np.max(retinex_image) - np.min(retinex_image))
retinex_image = (retinex_image * 255).astype(np.uint8)
cv2.imwrite(f'retinex_image_{img_path}', retinex_image)

True

In [58]:
def add_lighting_effect(image, center=None, radius=100, strength=1.5):
    if center is None:
        center = (image.shape[1] // 2, image.shape[0] // 2)
    
    mask = np.zeros_like(image, dtype=np.float32)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    mask = cv2.GaussianBlur(mask, (21, 21), 0)
    
    highlighted = cv2.addWeighted(image, 1, mask.astype(image.dtype), strength, 0)
    return highlighted

image = cv2.imread(img_path)
highlighted_image = add_lighting_effect(image)
cv2.imwrite(f'highlighted_image_{img_path}', highlighted_image)

True